In [1]:
import PIL
import cv2
# openCV bibliothèque pour le traitement des images et des vidéos

import numpy as np
#NumPy is used for working with arrays

import pandas as pd
#pandas is for  data analysis and manipulation tool

from PIL import Image as im
# python image library used for image processing. This library provides
#so many features for working on images using Python 
# from IPython import display
from tkinter import *
import tkinter as tk
from tkinter import filedialog , messagebox
import cv2
import os
from tkinter import PhotoImage
from PIL import ImageTk , Image
import numpy as np
import time

les classes

In [2]:
# Lire les coco names
classeFile = 'CocoNames.txt'
classesNames=[]
with open(classeFile , 'rt') as f :
    classesNames = f.read().rstrip('\n').split('\n')
# print(len(classesNames)) 
# print('{}'.format(classesNames[17]))

configuration de modele et la creation de Network

In [3]:
# configuration momdel 
modelConfig = 'yolov3.cfg'
modelWeights = 'yolov3.weights'
# create Network
net = cv2.dnn.readNetFromDarknet(modelConfig , modelWeights)
# use Opencv as the backend , and CPU
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
def detect_video(video_Path , save_video_pat):
#     video_Path = "./videoTeste/people.mp4"
    # def DetectVideo(video_Path):    
    cap = cv2.VideoCapture("dogs.mp4")
    # cap width and height
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # videoWriter object (copy the frames in outputVd)
    out = cv2.VideoWriter(save_video_pat , cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 10, (frame_width,frame_height))

    while(cap.isOpened()):
        ret, frame = cap.read()

        if (cv2.waitKey(55) == ord('q')):
            break  
        blob = cv2.dnn.blobFromImage(frame , 1/255 , (320 , 320) , [0,0,0] , 1 , crop = False)
        net.setInput(blob)
        LayersNames =net.getLayerNames()
        IndexOutLayers = np.array(net.getUnconnectedOutLayers()).reshape(-1 , 1)
        outLayersNames = [LayersNames[i[0]-1] for i in IndexOutLayers]
        outputs = net.forward(outLayersNames)
#       ------------ Evoque methode --------------
        findObjects(outputs , frame)
#         cv2.imshow("dog video" , frame)

        # Write the frame into the file 'outputVd.mp4'
        out.write(frame)
    # When everything done, release the video capture and video write objects
    out.release()    
    cap.release()
    cv2.destroyAllWindows()

In [5]:
v_conf = 0.5 # pour verifier l'existance de l'object 
def findObjects(outputs , img) :
    print("normal img" , img)
    hT , wT , cT = img.shape
    bbox = []
    classesIds = []
    confs = []
#     print("output" , outputs)
    for i in outputs :
        for j in i :
            scores = j[5:] # les probats des classes
            classId = np.argmax(scores) 
            confidence = scores[classId]
            if confidence > 0.5 :
                w , h = int(j[2]*wT) , int(j[3]*hT) 
                x , y = int((j[0]*wT)-w/2) , int((j[1]*hT) -h/2) 
                bbox.append([x , y , w , h])
                classesIds.append(classId)
                confs.append(float(confidence))
    indicesKeeping = cv2.dnn.NMSBoxes(bbox , confs , v_conf , nms_threshold = 0.3 ) # return indecices u should be keeped(more bbox decrease nmsValue)
    return [bbox , classesIds , confs ,indicesKeeping ]
#     print("indice keep" , indicesKeeping)
#     for i in indicesKeeping :
#         TrueBbox = bbox[i]
#         x , y , w , h =TrueBbox[:] 
#         print("x , y, w ,h" , x ,y , w ,h)
# #         draw the bbox as rectangle 
#         cv2.rectangle(img , (x , y) , (x+w , y+h) , (255 ,0 ,255) , 2)
#         print("img 2" , img)
# #       put object text classesIds[i] return l'indice de la classe de l'objet 
#         cv2.putText(img , " {} {}%".format(classesNames[classesIds[i]] ,int(confs[i]*100)) , (x , y-15),
#                     cv2.FONT_HERSHEY_SIMPLEX , 0.6 ,
#                             (255 , 0,255 ) , 2)
#         print("img 3 text puting" ,img)

In [6]:
def reel_time_detect(): 
    root = Tk()
    framTitre = Frame(root , bg='#2D2626')
    framTitre.place(x=5 , y=3 , width=1215, height=45)
    Label(framTitre , text=' Real time objects detection ' , font=('times' , 18 , 'bold') , fg='white', bg='#000000').pack(fill=X , pady=7)


    #------- fonction utilisees ---------
    def starting():
        try :
            global cap, img , tempe , compteur, _
            def mouse(event):# the event happend to the mouse 
                if event == cv2.EVENT_LBUTTONDBLCLK:
                    print('yes , its run ------')
            cap = cv2.VideoCapture(0)
            compteur = 1
            tempe = 0
            while True :
# #                     widthe  = cap.get(3)  # float `width`
# #                     height = cap.get(4)  # float `height`
                start = time.perf_counter() #represents the start time of my timer.
                print("start" , start)
                if (cv2.waitKey(2)== ord('q')):
                    break
                ret , img = cap.read()
                try :
#                     cv2.namedWindow('img')
                    cv2.setMouseCallback('img', mouse)
#                     if( cv2.setMouseCallback("img", mouse)==True):
#                         print("Left Mouse Button has been clicked")
                except Exception as e:
                    print('error in clicked mouse')
                    print('_________error name : ' , str(e)) 
                
                blob = cv2.dnn.blobFromImage(img , 1/255 , (320 , 320) , [0,0,0] , 1 , crop = False)
                #"1" scale factor applied to the image pixel values after normalization
                net.setInput(blob)
                LayersNames =net.getLayerNames()
                IndexOutLayers = np.array(net.getUnconnectedOutLayers()).reshape(-1 , 1)
                outLayersNames = [LayersNames[i[0]-1] for i in IndexOutLayers]
                outputs = net.forward(outLayersNames)
    #             print('output' , outputs)
                bbox , classesIds , confs , indicesKeeping = findObjects(outputs , img)[:]
    #             print('bbox : ' , bbox)
    #             print('indices : ',indicesKeeping)
                for i in indicesKeeping :
                    TrueBbox = bbox[i]
                    x , y , w , h =TrueBbox[:] 
            #         draw the bbox as rectangle 
                    cv2.rectangle(img , (x , y) , (x+w , y+h) , (255 ,0 ,255) , 2)
            #       put object text classesIds[i] return l'indice de la classe de l'objet 
                    cv2.putText(img , " {} {}%".format(classesNames[classesIds[i]] ,int(confs[i]*100)) , (x , y-15),
                                cv2.FONT_HERSHEY_SIMPLEX , 0.6 ,(255 , 0,255 ) , 2)
                end = time.perf_counter()
                tempe = tempe + (end - start)
                compteur+=1
                cv2.imshow('webcam' , img)
            cap.release()
            cv2.destroyAllWindows()
    #         print(tempe , compteur)
            txt = ("temps d'execution: "+ str(round(tempe,3))+" / temps de détecter une image:"+str(round(tempe/compteur , 3))+" sec.")
            _ = Label(FrmFooter1 , text= txt, font=('italic' , 15) , fg='black' , bg='orange')
            _.pack(side=LEFT)
            try:
                pathdir.destroy()
            except : 
                pass
        except :
            print("it's exception  ! ")

    def informe():
        try:
            _.destroy()
            pathdir.destroy()
        except:
            pass
        tk.messagebox.showinfo(title="informe" , message = "to quit the webcam, clic on : 'q' ")


    def strRecorde():
        try:
            _.destroy()
            pathdir.destroy()
        except:
            pass
        messagebox.showinfo(title='record path' , message="thanks, Select a path for the recording...")
        btnSvPath.pack(side=RIGHT , padx=2 , pady=2)


    def selectpath():
        global pathdir
        pathdir = ''
        dirictory = filedialog.askdirectory(title="Sélectionnez un répertoire de destination" , mustexist = True)
        name = tk.simpledialog.askstring(FrmVideo , "Choice a name for the video  of detection in real time : " )
        print("name chosed " , name)
        while(name == "" or name.isspace()):
            name = tk.simpledialog.askstring(FrmVideo , "the name must be string: " )
        save_video_pat = dirictory+"/"+name+".mp4" 
        pathdir=Label(FrmFooter2 , text='path : '+save_video_pat, font=('Arial' , 14 ) , fg='white' , bg='black')
        pathdir.pack(side=LEFT , padx=1)
    #  --------------- Recording video ---------------------------
        # video writter object
        capt = cv2.VideoCapture(0)
        frame_width = int(capt.get(3))
        frame_height = int(capt.get(4))
        outFrame = cv2.VideoWriter(save_video_pat , cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 10, (frame_width,frame_height))
        bbox_obj = []
        while(capt.isOpened()):
            if (cv2.waitKey(1)== ord('q')):
                break
            ret , FRAME = capt.read()
#             print("return" ,ret , "FRAME" , FRAME)
            # ---------------------------------
            blob = cv2.dnn.blobFromImage(FRAME , 1/255 , (320 , 320) , [0,0,0] , 1 , crop = False)
            net.setInput(blob)
            LayersNames =net.getLayerNames()
            IndexOutLayers = np.array(net.getUnconnectedOutLayers()).reshape(-1 , 1)
            outLayersNames = [LayersNames[i[0]-1] for i in IndexOutLayers]
            outputs = net.forward(outLayersNames)
#             print("Outputs ----" , outputs) run well
#             print("find funct result : "  ,findObjects(outputs , FRAME)[:])
            bbox , classesIds , confs , indicesKeeping = findObjects(outputs , FRAME)[:]
            print("bbox" , bbox , "\nClass id " , classesIds, "\nconfs" , confs , "\n indices keepin", indicesKeeping)
#             boite_index = 0
            for i in indicesKeeping :
                bbox_obj.append(bbox[i])
                TrueBbox = bbox[i]
                x , y , w , h =TrueBbox[:]
                cv2.rectangle(FRAME , (x , y) , (x+w , y+h) , (255 ,0 ,255) , 2)
                cv2.putText(FRAME , " {} {}%".format(classesNames[classesIds[i]] ,int(confs[i]*100)) , (x , y-15),
                            cv2.FONT_HERSHEY_SIMPLEX , 0.6 ,(255 , 0,255 ) , 2)
#                 if(((x - width/2) <= X_clique <= (x + width/2 )) and ((y - height/2 ) <= Y_clique <= (y+height/2))) :
#                     toSpeech.say(classesNames[classesIds[boite_index]])
#                     toSpeech.runAndWait()                          
#                 boite_index = boite_index+1    
#  --------------------- speech ---------------------
            outFrame.write(FRAME)
            cv2.imshow('webcam' , FRAME)
        outFrame.release()
        capt.release()
        cv2.destroyAllWindows()

#     def pointeur(event):
#         print("Clic détecté en X="  ,str(event.x) + "/  Y=" , str(event.y)

    def quitter():
        root.destroy()


    FrmBtns = Frame(root , bg='#4C93ED')#2288E7,4C93ED,
    FrmBtns.place(x=5 , y=50 , width=200, height=600)
    # icon 
    cameraIcon = PhotoImage(file = "F:/dell/Desktop/PFE/icons/camera.png")
    cameraIcon = cameraIcon.subsample(5,5)
    lblimg = Label(FrmBtns , image = cameraIcon, bg='#4C93ED')
    lblimg.pack(side=TOP , pady=50)
    #------ add buttons ---------
    starticon = PhotoImage(file = 'F:/dell/Desktop/PFE/icons/start.png')
    stopicon = PhotoImage(file = 'F:/dell/Desktop/PFE/icons/information2.png')
    recordicon = PhotoImage(file = 'F:/dell/Desktop/PFE/icons/recording.png')
    starticon = starticon.subsample(14,14)
    stopicon = stopicon.subsample(14,14)
    recordicon = recordicon.subsample(15,15)
    stopbtn = Button(FrmBtns , text=" information ",font =('Arial',15 , 'bold'),bd=5,cursor="hand2" , fg='white' , bg='#232121' , image=stopicon, compound=LEFT , command = informe)
    startbtn = Button(FrmBtns , text="  Start   ",font=('Arial',15 , 'bold'),bd=5,cursor="hand2" , fg='white' , bg='#232121' , image=starticon , compound=LEFT , command=starting)
    recordbtn = Button(FrmBtns , text=" Start/Record ",font =('Arial',15 , 'bold'),bd=4,cursor="hand2" , fg='white' , bg='#232121' , image=recordicon, compound=LEFT , command=strRecorde)
    stopbtn.pack(padx=2 , pady=15)
    startbtn.pack(padx=2 , pady=15)
    recordbtn.pack(padx=2 , pady=15)

    # ---- video impotation ----------
    global lbl , FrmVideo
    FrmVideo = Frame(root , bg='white')
    FrmVideo.place(x=213 , y=80 , width=1006, height=570)
#     _____________________________________________________________________________________
#     img.bind("<Button-1>", pointeur)
#        try :      cap.bind("<Button-1>", pointeur)  except :    print("  cap  Error  ")


    # the bottom
    global FrmFooter1,FrmFooter2 ,lbl_tempe,btnSvPath
    FrmFooter1 = Frame(root , bg='#264d84')
    FrmFooter1.place(x=5 , y=650 , width=606, height=45)
    lbl_tempe = Label(FrmFooter1 )#, fg='black' , bg='white')
    lbl_tempe.pack(side=LEFT , padx=1)
    FrmFooter2 = Frame(root , bg='#264d84')
    FrmFooter2.place(x=614 , y=650 , width=606, height=45)
    btnSvPath = Button(FrmFooter2 , text=' path ' , font=('Arial' , 12 , 'bold') ,cursor='hand2', fg='white' , bg='#f82c8b' , command=selectpath)
    #  ---------- frame btn Quitter -----------
    FrmQtt = Frame(root , bg='#264d84')#f82c8b
    FrmQtt.place(x=213 , y=50 , width=1006, height=45 )
    Button(FrmQtt , text='destroy', font=('Arial' , 13 ) , fg='white' , bg='red' ,cursor='hand2' ,command= quitter ).pack(side=RIGHT , pady=1 , padx=1)

    root.geometry("1225x900")
#     Button(root , text='cliqueeee' , command=pointeur).pack(side=RIGHT)
    root.title("Real time object detection")
    root.configure(bg='#2D2626')#D7DEDE
    root.resizable(0 ,0)
    root.mainloop()


In [ ]:
reel_time_detect()

start 27.9441956
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[ 64  18  20]
  [ 64  18  22]
  [ 64  17  24]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[ 62  17  14]
  [ 63  18  17]
  [ 63  17  19]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[ 60  17  14]
  [ 60  17  14]
  [ 60  17  14]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[ 90  43  29]
  [ 90  43  29]
  [ 91  44  30]
  ...
  [116  54  21]
  [118  58  22]
  [121  62  26]]

 [[ 90  43  29]
  [ 90  43  29]
  [ 90  43  29]
  ...
  [114  51  19]
  [116  55  22]
  [121  61  28]]

 [[ 92  45  31]
  [ 91  44  30]
  [ 91  44  30]
  ...
  [112  51  19]
  [117  56  23]
  [123  62  29]]]
start 29.4598209
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modul

normal img [[[ 20  13  16]
  [ 22  14  16]
  [ 22  15  15]
  ...
  [  5   0   1]
  [  4   0   2]
  [  4   0   2]]

 [[ 18  13  16]
  [ 18  13  15]
  [ 18  14  14]
  ...
  [  5   0   0]
  [  4   0   0]
  [  4   0   0]]

 [[ 17  14  17]
  [ 17  14  17]
  [ 16  13  16]
  ...
  [  4   0   0]
  [  4   0   0]
  [  4   0   0]]

 ...

 [[103  94  83]
  [ 98  90  80]
  [ 95  87  79]
  ...
  [  2  26  77]
  [ 11  34  84]
  [ 36  58 109]]

 [[112 105  95]
  [112 105  95]
  [113 106  98]
  ...
  [  6  27  82]
  [  7  25  83]
  [  3  22  79]]

 [[116 111 102]
  [116 111 102]
  [116 111 102]
  ...
  [  1  21  77]
  [  4  23  80]
  [  8  26  84]]]
start 35.0472943
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[ 53  64  72]
  [ 57  67  74]
  [ 58  65  74]
  ...
  [  6   3  17]
  [  8   6  17]
  [  8   6  17]]

 [[ 50 

normal img [[[ 95 103 119]
  [ 97 104 120]
  [100 105 121]
  ...
  [137 140 146]
  [137 140 146]
  [135 139 145]]

 [[ 94 102 117]
  [ 95 103 119]
  [ 97 104 120]
  ...
  [138 141 146]
  [137 140 144]
  [137 140 144]]

 [[ 95 103 119]
  [ 95 103 119]
  [ 97 104 120]
  ...
  [133 139 143]
  [133 139 143]
  [135 140 144]]

 ...

 [[  0   5  43]
  [  0   5  44]
  [  0   5  45]
  ...
  [130 138 142]
  [130 138 142]
  [131 140 143]]

 [[  0   4  44]
  [  0   3  42]
  [  0   3  41]
  ...
  [130 138 142]
  [131 140 143]
  [131 140 143]]

 [[  0   3  41]
  [  0   3  41]
  [  0   2  40]
  ...
  [130 138 142]
  [131 140 143]
  [131 140 143]]]
start 39.9304393
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[100 108 114]
  [ 98 105 111]
  [ 98 105 111]
  ...
  [131 136 143]
  [131 136 143]
  [131 136 143]]

 [[100 

normal img [[[120 127 135]
  [120 126 133]
  [121 126 132]
  ...
  [ 30  14  14]
  [ 29  14  20]
  [ 31  16  23]]

 [[120 127 133]
  [118 126 132]
  [118 126 132]
  ...
  [ 29  14  14]
  [ 29  15  18]
  [ 30  16  20]]

 [[121 126 132]
  [121 126 132]
  [121 126 132]
  ...
  [ 27  14  16]
  [ 27  15  18]
  [ 26  14  17]]

 ...

 [[160 163 170]
  [161 164 171]
  [162 165 172]
  ...
  [ 52  52  52]
  [ 51  51  51]
  [ 51  51  51]]

 [[164 163 170]
  [165 164 171]
  [165 164 171]
  ...
  [ 52  52  53]
  [ 51  51  53]
  [ 51  51  53]]

 [[165 164 171]
  [165 164 171]
  [165 164 171]
  ...
  [ 55  54  56]
  [ 54  53  55]
  [ 55  54  57]]]
start 49.0511179
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[135 129 132]
  [135 131 134]
  [134 130 135]
  ...
  [105  94  86]
  [111 101  93]
  [116 107  99]]

 [[131 

normal img [[[134 137 125]
  [133 138 125]
  [132 140 125]
  ...
  [134 131 140]
  [134 129 139]
  [134 129 139]]

 [[134 137 125]
  [132 137 125]
  [130 137 125]
  ...
  [131 131 138]
  [132 130 137]
  [132 130 137]]

 [[133 135 126]
  [133 135 126]
  [134 136 127]
  ...
  [127 131 135]
  [127 131 135]
  [129 132 136]]

 ...

 [[166 166 168]
  [168 167 169]
  [169 168 171]
  ...
  [ 41  41  36]
  [ 42  40  36]
  [ 41  39  35]]

 [[168 168 168]
  [168 168 168]
  [168 168 168]
  ...
  [ 41  39  32]
  [ 43  40  33]
  [ 43  40  33]]

 [[170 171 168]
  [168 168 167]
  [168 168 168]
  ...
  [ 43  38  32]
  [ 42  38  32]
  [ 41  38  32]]]
start 57.3018087
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[135 135 145]
  [134 137 146]
  [132 137 146]
  ...
  [131 129 135]
  [130 129 134]
  [130 129 134]]

 [[134 

...

normal img [[[ 80  98 119]
  [ 76  87 109]
  [ 74  78 101]
  ...
  [144 136 144]
  [145 136 144]
  [146 137 145]]

 [[ 79  97 116]
  [ 75  89 109]
  [ 70  79  99]
  ...
  [142 137 144]
  [143 137 144]
  [144 138 145]]

 [[ 77  98 116]
  [ 78  94 112]
  [ 71  83 101]
  ...
  [142 137 144]
  [143 137 144]
  [143 137 144]]

 ...

 [[ 62  69  96]
  [ 62  71  97]
  [ 61  71  97]
  ...
  [157 163 167]
  [155 161 165]
  [157 163 167]]

 [[ 53  62  91]
  [ 54  64  91]
  [ 55  65  91]
  ...
  [157 163 166]
  [158 164 168]
  [159 165 169]]

 [[ 49  58  89]
  [ 51  59  89]
  [ 52  61  90]
  ...
  [158 164 168]
  [159 165 169]
  [160 166 170]]]
start 156.9227901
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[ 79  85  87]
  [ 76  83  85]
  [ 72  78  82]
  ...
  [133 138 142]
  [130 139 140]
  [131 140 141]]

 [[ 78

normal img [[[137 132 146]
  [140 133 146]
  [149 141 153]
  ...
  [138 136 136]
  [138 136 136]
  [136 134 134]]

 [[136 134 145]
  [137 135 145]
  [142 140 149]
  ...
  [136 136 137]
  [135 135 137]
  [134 133 136]]

 [[134 137 146]
  [135 138 147]
  [138 140 149]
  ...
  [136 136 138]
  [135 135 137]
  [135 135 137]]

 ...

 [[158 163 170]
  [158 163 170]
  [158 163 170]
  ...
  [158 164 168]
  [158 164 168]
  [158 164 168]]

 [[156 164 168]
  [156 164 168]
  [156 164 168]
  ...
  [157 165 169]
  [157 165 169]
  [156 164 168]]

 [[156 164 168]
  [156 164 168]
  [156 164 168]
  ...
  [159 165 169]
  [158 164 168]
  [157 163 167]]]
start 165.130714
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[145 139 144]
  [139 134 138]
  [138 132 137]
  ...
  [128 133 150]
  [129 134 152]
  [128 133 151]]

 [[142 

normal img [[[123 124 122]
  [122 122 122]
  [121 121 123]
  ...
  [120 126 121]
  [119 125 118]
  [118 124 117]]

 [[126 124 120]
  [123 123 120]
  [121 121 121]
  ...
  [120 126 122]
  [119 125 120]
  [119 125 120]]

 [[126 124 120]
  [123 123 120]
  [121 121 121]
  ...
  [120 126 122]
  [119 125 120]
  [119 125 120]]

 ...

 [[ 51  59  65]
  [ 57  64  71]
  [ 65  73  79]
  ...
  [154 161 151]
  [154 161 151]
  [154 161 151]]

 [[ 57  62  66]
  [ 60  66  71]
  [ 67  72  79]
  ...
  [155 162 152]
  [155 162 152]
  [154 161 151]]

 [[ 64  68  72]
  [ 69  70  76]
  [ 71  69  76]
  ...
  [155 162 152]
  [155 162 152]
  [155 162 152]]]
start 177.2394928
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[126 124 131]
  [127 125 132]
  [128 126 133]
  ...
  [116 123 134]
  [116 123 134]
  [115 122 132]]

 [[123

normal img [[[135 136 133]
  [136 136 136]
  [137 137 139]
  ...
  [134 130 140]
  [133 130 142]
  [130 128 140]]

 [[132 135 132]
  [133 135 135]
  [134 136 138]
  ...
  [132 130 137]
  [132 129 139]
  [130 128 138]]

 [[133 133 131]
  [135 135 135]
  [136 136 138]
  ...
  [130 129 135]
  [130 129 136]
  [130 129 136]]

 ...

 [[168 168 168]
  [166 166 166]
  [166 166 166]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[168 168 168]
  [166 166 167]
  [166 166 166]
  ...
  [  0   0   0]
  [  0   0   2]
  [  0   0   2]]

 [[166 166 166]
  [166 166 167]
  [166 166 166]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
start 185.7244147
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[138 128 141]
  [137 129 140]
  [135 128 138]
  ...
  [116 128 132]
  [118 130 136]
  [119 131 137]]

 [[137

normal img [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 6]
  [0 0 9]
  [0 0 9]]

 [[0 1 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 6]
  [0 0 9]
  [0 0 9]]

 [[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 0 5]
  [0 0 7]
  [0 0 8]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
start 193.3364909
error in clicked mouse
_________error name :  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2558: error: (-27:Null pointer) NULL window: 'img' in function 'cvSetMouseCallback'

normal img [[[0 0 2]
  [0 0 2]
  [0 0 2]
  ...
  [0 0 2]
  [0 0 4]
  [0 0 4]]

 [[0 0 2]
  [0 0 2]
  [0 0 2]
  ...
  [0 0 1]
  [0 0 2]
  [0 0 2]]

 [[0 0 2]
  [0 0 2]
  [0 0 2]
  ...
  [0 0 1]
  [0 0 2]
  [0 0 2]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  